In [1]:
print("hello")

hello


In [1]:
import csv
import subprocess
import getpass
import os
import gzip
from os import listdir
from os.path import isfile, join
from SciServer import Authentication

myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
passwd = getpass.getpass('Password for ' + myUserName + ': ')
userstring = "username=" + myUserName + ",workgroup=win,uid=idies,password=" + passwd
projectname = input("Please enter the name of your SAFE Desktop project folder : ") 
dir = "//cloud.nas.jh.edu/sddesktop$/" + projectname
devnull = open(os.devnull, 'w')
subprocess.run(["sudo", "mkdir", "/home/idies/workspace/SAFE"], capture_output=False)
subprocess.run(["sudo", "chown", "idies:idies", "/home/idies/workspace/SAFE/"], capture_output=False)
try:
    subprocess.run(["sudo", "mount", "-t", "cifs", dir, "/home/idies/workspace/SAFE/", "-o", userstring], stdout=devnull, stderr=devnull)
except FileNotFoundError as e:
    print(e)

Password for mkim239:  ··········
Please enter the name of your SAFE Desktop project folder :  LCICM


In [5]:
# extracting patient_id of patients that have ECG recordings
# UPDATE: changing to patients that have 12-lead ECG recordings
patient_groups = [ f.path for f in os.scandir('SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/') if f.is_dir ]
print(patient_groups)

['SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1941', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1159', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1283', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1748', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1512', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1421', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1040', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1327', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1046', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1134', 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1569'

In [13]:
ecg_ids = []
for patient_group in patient_groups:
    for f in os.scandir(patient_group):
        if f.is_dir():
            ecg_ids.append(int(f.name[1:]))
print(len(ecg_ids))

160720


In [14]:
import pandas as pd

# opening relavant files
icd_codes = pd.read_csv('SAFE/Databases/mimic-iv-2.2/hosp/diagnoses_icd.csv')
icu_stays = pd.read_csv('SAFE/Databases/mimic-iv-2.2/icu/icustays.csv')
lab_events = pd.read_csv('SAFE/Databases/mimic-iv-2.2/hosp/labevents.csv')
lab_events_id = pd.read_csv('SAFE/Databases/mimic-iv-2.2/hosp/d_labitems.csv')
lab_events = pd.merge(lab_events, lab_events_id, on='itemid', how='left')
print ("File reading complete")

File reading complete


In [15]:
# total ICU population
print(len(icu_stays))

# ICU population w/ 24 hour data
icu_stays = icu_stays[icu_stays['los']>=1]
icu_stays=icu_stays.drop_duplicates('hadm_id')
icu_ids = list(icu_stays.hadm_id)
print(len(icu_ids)) 
print(icu_stays['subject_id'].nunique()) # e: 53034, p: 42264

73181
53034
42264


In [16]:
# filter every file with confirmed ICU stay
icd_codes = icd_codes[icd_codes['hadm_id'].isin(icu_ids)]
lab_events = lab_events[lab_events['hadm_id'].isin(icu_ids)]

In [17]:
# filter for HFpEF w/ ICD codes
hfpef_icd9 = icd_codes[icd_codes['icd_code'].str.startswith('4283')]
hfpef_icd10 = icd_codes[icd_codes['icd_code'].str.startswith('I503')]
hfpef_ids = pd.concat([hfpef_icd9, hfpef_icd10])
hfpef_ids = hfpef_ids.drop_duplicates('hadm_id')

hfpef_ids = hfpef_ids['hadm_id']
hfpef_ids = pd.merge(hfpef_ids, icu_stays, on='hadm_id', how="left")
# for encounter 
print(len(hfpef_ids))
# for patient 
print(hfpef_ids['subject_id'].nunique())

6116
4786


In [18]:
probnp_total = lab_events[(lab_events['itemid'] == 50963)]
probnp_total = probnp_total.dropna(subset=['valuenum'])
probnp_total = probnp_total.drop_duplicates('hadm_id')
probnp_total_ids  = list(probnp_total.hadm_id)
print(len(probnp_total_ids))

# filter for proBNP over and under 200
probnp_positive = probnp_total[probnp_total['valuenum'] > 200]
probnp_positive_ids = list(probnp_positive.hadm_id)
print(len(probnp_positive_ids))

# < 200
probnp_negative = probnp_total[probnp_total['valuenum'] <= 200]
probnp_negative_ids = list(probnp_negative.hadm_id)
print(len(probnp_negative_ids))

5542
5216
326


In [19]:
from datetime import datetime, timedelta

# HFpEF: proBNP + ICD codes
hfpef_icd_positive = hfpef_ids.drop('subject_id', axis=1)
hfpef_icd_positive = pd.merge(hfpef_icd_positive, probnp_positive, on='hadm_id', how="left")
hfpef_icd_positive = hfpef_icd_positive.dropna(subset=['valuenum'])

# convert to datetime object
def parse_date(str):
    return datetime.strptime(str, '%Y-%m-%d %H:%M:%S')
def get_date(datetime):
    return datetime.da

# set intime cutoff to 30 days before measurement
# filter additionally based on time
# convert to datetime objects
hfpef_icd_positive["intime"] = hfpef_icd_positive["intime"].apply(parse_date)
hfpef_icd_positive["outtime"] = hfpef_icd_positive["outtime"].apply(parse_date)
hfpef_icd_positive["charttime"] = hfpef_icd_positive["charttime"].apply(parse_date)
hfpef_icd_positive = hfpef_icd_positive[hfpef_icd_positive['charttime'] >= hfpef_icd_positive['intime'] - timedelta(days=30)]
hfpef_icd_positive = hfpef_icd_positive[hfpef_icd_positive['charttime'] <= hfpef_icd_positive['outtime']]

print("HFpEF_ICD_Positive encounter: " + str(len(hfpef_icd_positive)))
print("HFpEF_ICD_Positive patient: " + str(hfpef_icd_positive['subject_id'].nunique()))

# HFpEF: proBNP + ICD codes (false sample)
hfpef_icd_negative = hfpef_ids.drop('subject_id', axis=1)
hfpef_icd_negative = pd.merge(hfpef_icd_negative, probnp_negative, on='hadm_id', how="left")
hfpef_icd_negative = hfpef_icd_negative.dropna(subset=['valuenum'])

# filter additionally based on time
# convert to datetime objects
hfpef_icd_negative["intime"] = hfpef_icd_negative["intime"].apply(parse_date)
hfpef_icd_negative["outtime"] = hfpef_icd_negative["outtime"].apply(parse_date)
hfpef_icd_negative["charttime"] = hfpef_icd_negative["charttime"].apply(parse_date)
hfpef_icd_negative = hfpef_icd_negative[hfpef_icd_negative['charttime'] >= hfpef_icd_negative['intime'] - timedelta(days=30)]
hfpef_icd_negative = hfpef_icd_negative[hfpef_icd_negative['charttime'] <= hfpef_icd_negative['outtime']]
print("HFpEF_ICD_Negative encounter: " + str(len(hfpef_icd_negative)))
print("HFpEF_ICD_Negative patient: " + str(hfpef_icd_negative['subject_id'].nunique()))

# HFpEF: proBNP + ICD codes + ECG
hfpef_icd_ecg = hfpef_icd_positive[hfpef_icd_positive['subject_id'].isin(ecg_ids)]
print("HFpEF_ICD_Positve + ECG encounter: " + str(len(hfpef_icd_ecg)))
print("HFpEF_ICD_Positve + ECG patients: " + str(hfpef_icd_ecg['subject_id'].nunique()))

HFpEF_ICD_Positive encounter: 1224
HFpEF_ICD_Positive patient: 1157
HFpEF_ICD_Negative encounter: 44
HFpEF_ICD_Negative patient: 44
HFpEF_ICD_Positve + ECG encounter: 1149
HFpEF_ICD_Positve + ECG patients: 1085


In [20]:
# filter for HFrEF w/ ICD codes
hfref_icd9 = icd_codes[icd_codes['icd_code'].str.startswith('4282')]
hfref_icd10 = icd_codes[icd_codes['icd_code'].str.startswith('I502')]
hfref_ids = pd.concat([hfref_icd9, hfref_icd10])
hfref_ids = hfref_ids.drop_duplicates('hadm_id')
# for encounter 
print(len(hfref_ids))
# for patient 
print(hfref_ids['subject_id'].nunique())
hfref_ids = hfref_ids['hadm_id']
hfref_ids = pd.merge(hfref_ids, icu_stays, on='hadm_id', how="left")

5924
4733


In [21]:
# HFrEF: proBNP + ICD codes
hfref_icd_positive = hfref_ids.drop('subject_id', axis=1)
hfref_icd_positive = pd.merge(hfref_icd_positive, probnp_positive, on='hadm_id', how="left")
hfref_icd_positive = hfref_icd_positive.dropna(subset=['valuenum'])

# filter additionally based on time
hfref_icd_positive["intime"] = hfref_icd_positive["intime"].apply(parse_date)
hfref_icd_positive["outtime"] = hfref_icd_positive["outtime"].apply(parse_date)
hfref_icd_positive["charttime"] = hfref_icd_positive["charttime"].apply(parse_date)
hfref_icd_positive = hfref_icd_positive[hfref_icd_positive['charttime'] >= hfref_icd_positive['intime'] - timedelta(days=30)]
hfref_icd_positive = hfref_icd_positive[hfref_icd_positive['charttime'] <= hfref_icd_positive['outtime']]
print("HFrEF_ICD_Positive encounter: " + str(len(hfref_icd_positive)))
print("HFrEF_ICD_Positive patient: " + str(hfref_icd_positive['subject_id'].nunique()))


# HFrEF: proBNP + ICD codes (false sample)
hfref_icd_negative = hfref_ids.drop('subject_id', axis=1)
hfref_icd_negative = pd.merge(hfref_icd_negative, probnp_negative, on='hadm_id', how="left")
hfref_icd_negative = hfref_icd_negative.dropna(subset=['valuenum'])

# filter additionally based on time
hfref_icd_negative["intime"] = hfref_icd_negative["intime"].apply(parse_date)
hfref_icd_negative["outtime"] = hfref_icd_negative["outtime"].apply(parse_date)
hfref_icd_negative["charttime"] = hfref_icd_negative["charttime"].apply(parse_date)
hfref_icd_negative = hfref_icd_negative[hfref_icd_negative['charttime'] >= hfref_icd_negative['intime'] - timedelta(days =30)]
hfref_icd_negative = hfref_icd_negative[hfref_icd_negative['charttime'] <= hfref_icd_negative['outtime']]
print("HFrEF_ICD_Negative encounter: " + str(len(hfref_icd_negative)))
print("HFrEF_ICD_Negative patient: " + str(hfref_icd_negative['subject_id'].nunique()))

# HFrEF: proBNP + ICD codes + ECG
hfref_icd_ecg = hfref_icd_positive[hfref_icd_positive['subject_id'].isin(ecg_ids)]
print("HFrEF_ICD_Positve + ECG encounter: " + str(len(hfref_icd_ecg)))
print("HFrEF_ICD_Positve + ECG patients: " + str(hfref_icd_ecg['subject_id'].nunique()))

HFrEF_ICD_Positive encounter: 1094
HFrEF_ICD_Positive patient: 1033
HFrEF_ICD_Negative encounter: 13
HFrEF_ICD_Negative patient: 13
HFrEF_ICD_Positve + ECG encounter: 1017
HFrEF_ICD_Positve + ECG patients: 957


In [22]:
# all ICD patients
chf_codes = pd.concat([hfpef_ids, hfref_ids])
chf_codes = chf_codes.drop_duplicates('hadm_id')
chf_codes_ids = list(chf_codes.hadm_id)
print(len(chf_codes))

icu_notin_icd = icu_stays[~icu_stays['hadm_id'].isin(chf_codes_ids)]

# no ICD but has proBNP
icu_has_probnp = icu_notin_icd[icu_notin_icd['hadm_id'].isin(probnp_total_ids)]
print("No ICD but has proBNP encounters: " + str(len(icu_has_probnp)))
print("No ICD but has proBNP patients: " + str(icu_has_probnp['subject_id'].nunique()))

# no ICD but proBNP >200
icu_probnp_positive = icu_has_probnp.drop('subject_id', axis=1)
icu_probnp_positive = pd.merge(icu_probnp_positive, probnp_positive, on='hadm_id', how="left")
icu_probnp_positive = icu_probnp_positive.dropna(subset=['valuenum'])
# filter additionally based on time
icu_probnp_positive = icu_probnp_positive[icu_probnp_positive['charttime'] >= icu_probnp_positive['intime']]
icu_probnp_positive = icu_probnp_positive[icu_probnp_positive['charttime'] <= icu_probnp_positive['outtime']]
print("No ICD + proBNP encounters (+): " + str(len(icu_probnp_positive)))
print("No ICD + proBNP patients (+): " + str(icu_probnp_positive['subject_id'].nunique())) # e: 1041, p: 1031

# no ICD but proBNP > 200 + ECGs
icu_probnp_positive_ecg = icu_probnp_positive[icu_probnp_positive['subject_id'].isin(ecg_ids)]
print("No ICD + proBNP (+) + ECG encounters: " + str(len(icu_probnp_positive_ecg)))
print("No ICD + proBNP (+) + ECG patients: " + str(icu_probnp_positive_ecg['subject_id'].nunique())) # e: 3, p: 3

# no ICD but proBNP >200
icu_probnp_negative = icu_has_probnp.drop('subject_id', axis=1)
icu_probnp_negative = pd.merge(icu_probnp_negative, probnp_negative, on='hadm_id', how="left")
icu_probnp_negative = icu_probnp_negative.dropna(subset=['valuenum'])
# filter additionally based on time
icu_probnp_negative = icu_probnp_negative[icu_probnp_negative['charttime'] >= icu_probnp_negative['intime']]
icu_probnp_negative = icu_probnp_negative[icu_probnp_negative['charttime'] <= icu_probnp_negative['outtime']]
print("No ICD + proBNP encounters (-): " + str(len(icu_probnp_negative)))
print("No ICD + proBNP patients (-): " + str(icu_probnp_negative['subject_id'].nunique())) # e: 119, p: 119
      
# no ICD but proBNP < 200 + ECGs
icu_probnp_negative_ecg = icu_probnp_negative[icu_probnp_negative['subject_id'].isin(ecg_ids)]
print("No ICD + proBNP (-) + ECG encounters: " + str(len(icu_probnp_negative_ecg)))
print("No ICD + proBNP (-) + ECG patients: " + str(icu_probnp_negative_ecg['subject_id'].nunique())) # e: 1, p: 1

12015
No ICD but has proBNP encounters: 2662
No ICD but has proBNP patients: 2591
No ICD + proBNP encounters (+): 1041
No ICD + proBNP patients (+): 1031
No ICD + proBNP (+) + ECG encounters: 953
No ICD + proBNP (+) + ECG patients: 943
No ICD + proBNP encounters (-): 119
No ICD + proBNP patients (-): 119
No ICD + proBNP (-) + ECG encounters: 107
No ICD + proBNP (-) + ECG patients: 107


In [25]:
# no ICD AND no proBNP
icu_no_icd_no_probnp = icu_notin_icd[~icu_notin_icd['hadm_id'].isin(probnp_total_ids)]
icu_no_icd_no_probnp = icu_no_icd_no_probnp.drop_duplicates("hadm_id")
print("ICU + no ICD + no proBNP encounters: " + str(len(icu_no_icd_no_probnp)))
print("ICU + no ICD + no proBNP patients: " + str(icu_no_icd_no_probnp['subject_id'].nunique())) # e: 38357, p: 32690 

# no ICD AND no proBNP + ECGs
icu_no_icd_no_probnp_ecg = icu_no_icd_no_probnp[icu_no_icd_no_probnp['subject_id'].isin(ecg_ids)]
print("ICU + no ICD + no proBNP + ECG encounters: " + str(len(icu_no_icd_no_probnp_ecg)))
print("ICU + no ICD + no proBNP + ECG patients: " + str(icu_no_icd_no_probnp_ecg['subject_id'].nunique())) # e: 76, p: 67

ICU + no ICD + no proBNP encounters: 38357
ICU + no ICD + no proBNP patients: 32690
ICU + no ICD + no proBNP + ECG encounters: 33961
ICU + no ICD + no proBNP + ECG patients: 28468


In [ ]:
# export csvs for feature extraction

hfpef_icd_ecg.to_csv("hfpef_icd_ecg.csv", sep=',', index=False, encoding='utf-8')
hfref_icd_ecg.to_csv("hfref_icd_ecg.csv", sep=',', index=False, encoding='utf-8')
icu_probnp_positive_ecg.to_csv("icu_probnp_positive_ecg.csv", sep=',', index=False, encoding='utf-8')
icu_probnp_negative_ecg.to_csv("icu_probnp_negative_ecg.csv", sep=',', index=False, encoding='utf-8')
icu_no_icd_no_probnp_ecg.to_csv("icu_no_icd_no_probnp_ecg.csv", sep=',', index=False, encoding='utf-8')